<a href="https://colab.research.google.com/github/jstarace/CS_504_Assignment3_Vakanski/blob/main/CS504_Vakanski_Assignment_3_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

### The following is a complete walk-through of Assignment #3 for CS 504: ST Adversarial Machine Learning, taught by Dr. Vakanski at the University of Idaho in the Spring of 2023.  The assignment is being made available to assist in the understanding of future related assignments at the request of Dr. Vakanski.


## The assignment is broken down into 2 parts.
1. Adversarial Traning defense against evasion attacks
1. Poisoning attacks against deep learning classification models

For this assignment I used Google Colab for the training of the models and completion of the assignment. It is recommend that the same or similar is used.


# Notebook Setup

## Setup and Configuration

Before diving into the assignment we want to make sure our environment has all the necessary libraries and connections

In [1]:
# Connect to Google Drive.
# All images required for the assignment were either provided by Dr. Vakanski or were publicly available
# Images were loaded onto google drive for ease of access

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install the adversarial robustness toolbox so that we can work with adversarial models
!pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


## Imports

In [3]:
# For inline matplot use
%matplotlib inline

# Import the following libraries

import os
import cv2
import sys
import keras
import random
import imageio
import tarfile
import natsort
import warnings
import numpy as np
import tensorflow as tf
import keras.backend as k
import matplotlib.pyplot as plt

In [4]:
# From installed and available libraries import these specific classes

from os import listdir

from keras.models import Model
from keras.applications import vgg16
from keras.preprocessing import image
from keras.layers import Dense, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping 
from keras.applications.imagenet_utils import preprocess_input

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.tools.docs.doc_controls import T
from tensorflow.keras.utils import to_categorical
from tensorflow.python.ops.math_ops import TruncateDiv
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, GlobalAveragePooling2D

from art.estimators.classification import KerasClassifier
from art.estimators.classification import SklearnClassifier

from art.attacks.evasion import BoundaryAttack
from art.attacks.evasion import FastGradientMethod,ProjectedGradientDescent,DeepFool
from art.utils import to_categorical

from skimage.util import compare_images
from IPython.display import clear_output
from matplotlib.gridspec import GridSpec
from skimage import data, transform, exposure

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from re import U

In [5]:
# This line is important, without it the attacks in the ART toolbox won't work
# And it needs to be run before training the models
tf.compat.v1.disable_eager_execution()

# Supress warnings
warnings.filterwarnings('ignore')

In [6]:
# Print the version of tensorflow and keras
print("TensorFlow version:{}".format(tf.__version__))
print("Keras version:{}".format(keras.__version__))

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

TensorFlow version:2.12.0
Keras version:2.12.0
Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


## Definitions

The following are the definitions for methods we will use during the assignment

### Plotting loss and accuracy

In [7]:
# Simple method used to print a models accuracy and loss.  Pass the method
# the history and both accuracy and loss will be displayed

def plot_accuracy_loss(the_history):
    # plot the accuracy and loss
    train_loss = the_history.history['loss']
    val_loss = the_history.history['val_loss']
    acc = the_history.history['accuracy'] 
    val_acc = the_history.history['val_accuracy']

    epochsn = np.arange(1, len(train_loss)+1,1)
    plt.figure(figsize=(12, 4))

    plt.subplot(1,2,1)
    plt.plot(epochsn, acc, 'b', label='Training Accuracy')
    plt.plot(epochsn, val_acc, 'r', label='Validation Accuracy')
    plt.grid(color='gray', linestyle='--')
    plt.legend()            
    plt.title('ACCURACY')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')

    plt.subplot(1,2,2)
    plt.plot(epochsn,train_loss, 'b', label='Training Loss')
    plt.plot(epochsn,val_loss, 'r', label='Validation Loss')
    plt.grid(color='gray', linestyle='--')
    plt.legend()
    plt.title('LOSS')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

### Model Definitions

In [8]:
# For this assignment transfer learning was used for the base models.
# Specifically resnet models were used.  You may note that when using the
# models provided by Keras, their documentation mentions preprocessing
# the images.  This is intentionally left out as it caused issues with the
# accuracy of the predicitions.

# As the inputs and classes may change we pass those as variables to provide a
# more robust method for use.
def build_vgg16(input_shape, class_count):

  base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

  # Add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  # Add a fully-connected layer
  x = Dense(2048, activation='relu')(x)
  x = Dropout(0.25)(x)
  x = Dense(256, activation='relu')(x)
  x = Dropout(0.25)(x)
  # Add a softmax layer 
  predictions = Dense(class_count, activation='softmax')(x)

  # The model 
  model = Model(inputs=base_model.input, outputs=predictions)

  return model

### Helper methods

In [9]:
# Generate an array of unique random ints used for selecting random indices
def random_array(c, upper):
  random_list = []
  while len(random_list) < c:
    #for i in range(c):
    r = random.randint(0, upper-1)
    if r not in random_list:
      random_list.append(r)
  return random_list

# Get a subset from provided array
def get_subset(subset, provided_images, provided_labels):
  temp_img = []
  temp_labels = []
  for x in subset:
    temp_img.append(provided_images[x])
    temp_labels.append(provided_labels[x])
  temp_img = np.array(temp_img, dtype = np.float32)
  temp_labels = np.array(temp_labels, dtype = np.float32)
  return temp_img, temp_labels

### Image Loader

In [10]:
def load_images(directory):

    imgs_list = []

    # List of all images in the directory
    imgs_list_1 = listdir(directory)
    # Make sure that the images are sorted 
    imagesList = natsort.natsorted(imgs_list_1)

    # Read the images as numpy arrays
    for i in range(len(imagesList)):
          tmp_img = cv2.imread(os.path.join(directory, imagesList[i]))
          img_arr = np.array(tmp_img)
          imgs_list.append(img_arr/255.)
    
    # Convert the lists to numpy arrays
    imgs = np.asarray(imgs_list)

    return imgs

### Global Variables

In [11]:
# Label Encoder for loading in data
le = LabelEncoder()

# Define a save path for use when training models
save_path = '/content/drive/MyDrive/Colab Notebooks/CS_504/Assignment_3/Saves/'

# Define the path for weights to be stored
weight_path = 'Weights/'

# Define the path for the model to be stored
model_path = 'Models/'

# Define the extention for weights/models
ext = '.h5'

# Part 2

## Step 1

**About the data**

Add this

### Load Data

In [ ]:
# Uncompress the dataset 
!unzip -uq "/content/drive/MyDrive/Colab Notebooks/CS_504/Assignment_3/data/BUS_images.zip" -d "sample_data/"

In [ ]:
# Load the images
all_images = load_images('sample_data/BUS_images')
print('Shape of the images:', all_images.shape)

# Load the labels
all_labels = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/CS_504/Assignment_3/data/labels_BUSimages.csv', delimiter=',', dtype=str)
image_labels = le.fit_transform(all_labels)
print('Shape of the labels:', image_labels.shape)

In [ ]:
# Split the data into 2 sets of data
# 1. A trainval set that will later be split into the training and validation sets
# 2. A test set which will be 20% of all data

# By setting the random state, we can ensure consistency when creating the datasets
# This allows us to focus on fine tuning our models
trainval_images, test_images, trainval_labels, test_labels = train_test_split(all_images, image_labels, test_size=0.2, random_state=12)

# Using the trainval sets, we split the data again into our training and validation sets
# The validation set will be 20% of the remaining data and the remainder will be 
# our training set
train_images, val_images, train_labels, val_labels = train_test_split(trainval_images, trainval_labels, test_size=0.2, random_state=12)

# With all the necessary info loaded, we can free up space by deleting the imported images
del all_images

In [ ]:
# Display the shapes of train, validation, and test datasets
print('Images train shape: {} - Labels train shape: {}'.format(train_images.shape, train_labels.shape))
print('Images validation shape: {} - Labels validation shape: {}'.format(val_images.shape, val_labels.shape))
print('Images test shape: {} - Labels test shape: {}'.format(test_images.shape, test_labels.shape))

# Display the range of images 
print('\nMax pixel value', np.max(train_images))
print('Min pixel value', np.min(train_images))
print('Average pixel value', np.mean(train_images))
print('Data type', train_images[0].dtype)

# A list with the names of the image classes
label_names = np.unique(all_labels)

del all_labels

### Display images

In [ ]:
# Plot a few images to check if the labels make sense
plt.figure(figsize=(9, 9))
plt.rcParams.update({'font.size': 8})
for n in range(9):
    i = np.random.randint(0, len(train_images), 1)
    ax = plt.subplot(3, 3, n+1)
    plt.imshow(train_images[i[0]])
    plt.title('Label: ' + str(label_names[train_labels[i[0]]]))
    plt.axis('off')

### Train a deep learning model above 80% accuracy

#### Paths and variables

In [ ]:
weight_name = 'bd_weights_'
bd = 'bd/'

In [ ]:
input = (224, 224, 3)
NUM_CLASSES = len(label_names)
batch = 32
epochs = 100
iterations = 1

In [ ]:
adam = tf.keras.optimizers.legacy.Adam(learning_rate = 1e-5)

model_monitor = EarlyStopping(monitor='val_loss',
                            min_delta=1e-3,
                            patience=50,
                            verbose=0,
                            mode='auto')

#### Train the model

In [ ]:
accuracy = []
histories = []

for i in range(iterations):
  model = build_vgg19(input, NUM_CLASSES)
  model.compile(loss="sparse_categorical_crossentropy",
              optimizer = adam,
              metrics=['accuracy'])
  print("Iteration: " + str(i+1) + " of " + str(iterations))
  path = save_path + weight_path + weight_name + str(i) + ext
  print("Weights will be saved here: " + path)
  model_checkpointer = ModelCheckpoint(path,
                                       verbose = 1,
                                       save_best_only = True,
                                       save_weights_only = True,
                                       mode = 'auto')
  history = model.fit(
      train_images,
      train_labels,
      verbose=1,
      epochs = epochs,
      batch_size = batch,
      callbacks = [model_monitor, model_checkpointer],
      validation_data = (val_images, val_labels)
  )

  histories.append(history)
  loss_test, accuracy_test = model.evaluate(test_images, test_labels)
  print('Accuracy on test data: {:4.2f}%'.format(accuracy_test * 100))
  accuracy.append(accuracy_test)

#### Validate the best model and load it

In [ ]:
## Print the index with the best accuracy
best = [index for index, item in enumerate(accuracy) if item ==max(accuracy)]
the_value = best[0]
print("The best index is: " + str(the_value))

In [ ]:
best_model = build_vgg16(input, NUM_CLASSES)
best_model.compile(loss="sparse_categorical_crossentropy",
                optimizer=adam,
                metrics=['accuracy'])

In [ ]:
best_path = save_path + weight_path + weight_name + str(the_value) + ext
best_model.load_weights(best_path)
best_model.summary()
plot_accuracy_loss(histories[the_value])
best_loss_test, best_accuracy_test = best_model.evaluate(test_images, test_labels)
print('Accuracy on test data: {:4.2f}%'.format(best_accuracy_test * 100))

#### Save best model
This allows us to skip retraining the model should we need to come back to this at a later time

In [ ]:
best_model.save(save_path + model_path + bd)